In [24]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import itertools

import geopy
import geopy.distance
import networkx as nx
import osmnx as ox
ox.config(log_console=True)

import folium
from geopy.geocoders import Nominatim

import time

In [25]:
place = 'Istanbul, Turkey'
G = ox.graph_from_place(place, network_type='drive')

In [26]:
addresses = ["Sapphire, Kağıthane, İstanbul", "İçmeler, Tuzla, İstanbul", "Cendere Caddesi, Kağıthane, İstanbul", 
             "Sabiha Gökçen Havalimanı, İstanbul", "Arnavutköy, İstanbul"]

In [27]:
def calculate_lowest_path(addresses):
    coordinates = [ox.geocoder.geocode(address) for address in addresses]
    nodes = [ox.nearest_nodes(G, coord[1], coord[0]) for coord in coordinates]

    all_permutations = itertools.permutations(nodes)

    lowest_distance = float('inf')
    lowest_permutation = []

    for perm in all_permutations:
        total_distance = 0
        for i in range(len(perm) - 1):
            start = perm[i]
            end = perm[i + 1]
            path_length = nx.shortest_path_length(G, source=start, target=end, weight='length')
            total_distance += path_length
        if total_distance < lowest_distance:
            lowest_distance = total_distance
            lowest_permutation = perm
    print("Route:")
    print("------")
    for i in range(len(lowest_permutation) - 1):
        start = lowest_permutation[i]
        end = lowest_permutation[i + 1]
        start_index = nodes.index(start)
        end_index = nodes.index(end)
        start_address = addresses[start_index]
        end_address = addresses[end_index]
        path_length = nx.shortest_path_length(G, source=start, target=end, weight='length')

        print(f"{start_address} to {end_address}: {round(path_length/1000,2)} km")
        
    print("---------------------------------------------------")    
    print(f"Total Distance: {round(lowest_distance/1000,2)} km")
    print("---------------------------------------------------")    
    print(f"Time(mins) to take with avg. of 100 km/h pace: {round(round(lowest_distance/1000,2)/100*60,2)} mins")
    map_center = [41, 29]

    m = folium.Map(location=map_center, zoom_start=10)

    for i in range(len(lowest_permutation) - 1):
        start = lowest_permutation[i]
        end = lowest_permutation[i + 1]
        start_index = nodes.index(start)
        end_index = nodes.index(end)
        start_address = addresses[start_index]
        end_address = addresses[end_index]
        route = nx.shortest_path(G, source=start, target=end, weight='length')
        route_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
        color = 'blue' if i % 2 == 0 else 'red'
        folium.PolyLine(route_coordinates, color=color, weight=4).add_to(m)

        folium.Marker(route_coordinates[0], icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(route_coordinates[-1], icon=folium.Icon(color='red')).add_to(m)

        folium.Marker(route_coordinates[0], icon=folium.DivIcon(
            html=f'<div style="font-weight: bold; font-size: 14px;">{start_address}</div>')).add_to(m)
        folium.Marker(route_coordinates[-1], icon=folium.DivIcon(
            html=f'<div style="font-weight: bold; font-size: 14px;">{end_address}</div>')).add_to(m)

    return m

# Trials

In [21]:
# 5 places:
start = time.time()
addresses = ["Sapphire, Kağıthane, İstanbul", "İçmeler, Tuzla, İstanbul", "Cendere Caddesi, Kağıthane, İstanbul", 
             "Sabiha Gökçen Havalimanı, İstanbul", "Arnavutköy, İstanbul"]
map1 = calculate_lowest_path(addresses)
end = time.time()

print("---------------------")
print("Run time mins: ", round((end - start)/60,2))
      
map1

Route:
------
Sabiha Gökçen Havalimanı, İstanbul to İçmeler, Tuzla, İstanbul: 9.7 km
İçmeler, Tuzla, İstanbul to Arnavutköy, İstanbul: 40.23 km
Arnavutköy, İstanbul to Sapphire, Kağıthane, İstanbul: 4.57 km
Sapphire, Kağıthane, İstanbul to Cendere Caddesi, Kağıthane, İstanbul: 3.87 km
---------------------------------------------------
Total Distance: 58.36 km
---------------------------------------------------
Time(mins) to take with avg. of 100 km/h pace: 35.02 mins
---------------------
Run time mins:  3.5


In [28]:
# 4 places :
start = time.time()
addresses = ["Sapphire, Kağıthane, İstanbul", "İçmeler, Tuzla, İstanbul", "Cendere Caddesi, Kağıthane, İstanbul", 
             "Sabiha Gökçen Havalimanı, İstanbul"]
map1 = calculate_lowest_path(addresses)
end = time.time()

print("---------------------")
print("Run time mins: ", round((end - start)/60,2))

map1

Route:
------
Cendere Caddesi, Kağıthane, İstanbul to Sapphire, Kağıthane, İstanbul: 3.62 km
Sapphire, Kağıthane, İstanbul to İçmeler, Tuzla, İstanbul: 39.55 km
İçmeler, Tuzla, İstanbul to Sabiha Gökçen Havalimanı, İstanbul: 9.18 km
---------------------------------------------------
Total Distance: 52.35 km
---------------------------------------------------
Time(mins) to take with avg. of 100 km/h pace: 31.41 mins
---------------------
Run time mins:  0.72


In [23]:
# 3 places:
start = time.time()
addresses = ["Sapphire, Kağıthane, İstanbul", "İçmeler, Tuzla, İstanbul", "Cendere Caddesi, Kağıthane, İstanbul"]
map1 = calculate_lowest_path(addresses)
end = time.time()

print("---------------------")
print("Run time mins: ", round((end - start)/60,2))

map1

Route:
------
İçmeler, Tuzla, İstanbul to Sapphire, Kağıthane, İstanbul: 39.14 km
Sapphire, Kağıthane, İstanbul to Cendere Caddesi, Kağıthane, İstanbul: 3.87 km
---------------------------------------------------
Total Distance: 43.01 km
---------------------------------------------------
Time(mins) to take with avg. of 100 km/h pace: 25.81 mins
---------------------
Run time mins:  0.17
